In [1]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

from tqdm import tqdm
from random import randint

plt.rcParams["figure.figsize"] = (20, 16)
warnings.simplefilter("ignore")
[i for i in os.listdir() if i.endswith(".csv") ]


### <8 mbits en cumsum = bruit 

['messenger20221117_125310.csv',
 'output.csv',
 'twitch20221117_113554.csv',
 'whats.csv',
 'youtube20221108_165214_raw.csv',
 'youtube20221108_182416.csv',
 'youtube20221108_182416_raw.csv',
 'yt.csv',
 'yt_whats.csv']

In [36]:
name_dataset = "dataset_images"
global_labels = ["noise", "whatsapp","youtube", "twitch","messenger"]
try :
    os.mkdir(name_dataset)
except :
    pass
for label in global_labels :
    try : 
        cur_path = os.path.join(name_dataset,label)
        os.mkdir(cur_path)
    except : pass

In [60]:
current_file = "twitch20221117_113554.csv"
current_file = "messenger20221117_125310.csv"
data = pd.read_csv(current_file)
#data = pd.read_csv('output.csv')

data

,Time,rnti,SFN,format,preamble,ta,nb_TB,MCSi_1,MCSi_2,TBS_1,TBS_2,nb_PRB,connection_id,acquisition
0,2022-11-17 11:53:53.714,62,2725,0.0,31.0,3.0,1,12,-2,376,-2,2,4,noise20221117_125310
1,2022-11-17 11:53:53.722,62,2733,2.0,31.0,3.0,1,14,-2,224,-2,1,4,noise20221117_125310
2,2022-11-17 11:53:53.724,62,2735,2.0,31.0,3.0,1,14,-2,224,-2,1,4,noise20221117_125310
3,2022-11-17 11:53:53.734,62,2745,0.0,31.0,3.0,1,12,-2,376,-2,2,4,noise20221117_125310
4,2022-11-17 11:53:53.762,62,2773,2.0,31.0,3.0,1,14,-2,224,-2,1,4,noise20221117_125310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373015,2022-11-17 12:41:04.253,75,7024,0.0,5.0,3.0,1,10,-2,328,-2,2,17,noise20221117_125310
373016,2022-11-17 12:41:04.261,75,7032,2.0,5.0,3.0,1,14,-2,224,-2,1,17,noise20221117_125310
373017,2022-11-17 12:41:04.287,75,7058,2.0,5.0,3.0,1,14,-2,488,-2,2,17,noise20221117_125310
373018,2022-11-17 12:41:04.313,75,7084,0.0,5.0,3.0,1,10,-2,328,-2,2,17,noise20221117_125310


In [61]:
first_acq = data.acquisition.iloc[0]
id_acquisition = first_acq[first_acq.find("202"):]
data2 = data.rename(columns = {"acquisition" : "label", "connection_id" : "acquisition"})
data2["acquisition"] = data2["acquisition"].apply(lambda x : f"{id_acquisition}_{x}")
data2["label"] = data2["label"].apply(lambda x : x.replace(id_acquisition, ""))
data2

,Time,rnti,SFN,format,preamble,ta,nb_TB,MCSi_1,MCSi_2,TBS_1,TBS_2,nb_PRB,acquisition,label
0,2022-11-17 11:53:53.714,62,2725,0.0,31.0,3.0,1,12,-2,376,-2,2,20221117_125310_4,noise
1,2022-11-17 11:53:53.722,62,2733,2.0,31.0,3.0,1,14,-2,224,-2,1,20221117_125310_4,noise
2,2022-11-17 11:53:53.724,62,2735,2.0,31.0,3.0,1,14,-2,224,-2,1,20221117_125310_4,noise
3,2022-11-17 11:53:53.734,62,2745,0.0,31.0,3.0,1,12,-2,376,-2,2,20221117_125310_4,noise
4,2022-11-17 11:53:53.762,62,2773,2.0,31.0,3.0,1,14,-2,224,-2,1,20221117_125310_4,noise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373015,2022-11-17 12:41:04.253,75,7024,0.0,5.0,3.0,1,10,-2,328,-2,2,20221117_125310_17,noise
373016,2022-11-17 12:41:04.261,75,7032,2.0,5.0,3.0,1,14,-2,224,-2,1,20221117_125310_17,noise
373017,2022-11-17 12:41:04.287,75,7058,2.0,5.0,3.0,1,14,-2,488,-2,2,20221117_125310_17,noise
373018,2022-11-17 12:41:04.313,75,7084,0.0,5.0,3.0,1,10,-2,328,-2,2,20221117_125310_17,noise


In [62]:
for val in data2.acquisition.unique():
    current_data = data2[data2.acquisition==val]
    print(current_data.acquisition.unique())
    print(current_data.acquisition.value_counts(),"\n")
    #plt.plot()

['20221117_125310_4']
20221117_125310_4    364319
Name: acquisition, dtype: int64 

['20221117_125310_5']
20221117_125310_5    11
Name: acquisition, dtype: int64 

['20221117_125310_6']
20221117_125310_6    11
Name: acquisition, dtype: int64 

['20221117_125310_8']
20221117_125310_8    1230
Name: acquisition, dtype: int64 

['20221117_125310_7']
20221117_125310_7    1229
Name: acquisition, dtype: int64 

['20221117_125310_9']
20221117_125310_9    628
Name: acquisition, dtype: int64 

['20221117_125310_10']
20221117_125310_10    580
Name: acquisition, dtype: int64 

['20221117_125310_11']
20221117_125310_11    1632
Name: acquisition, dtype: int64 

['20221117_125310_12']
20221117_125310_12    1486
Name: acquisition, dtype: int64 

['20221117_125310_13']
20221117_125310_13    578
Name: acquisition, dtype: int64 

['20221117_125310_14']
20221117_125310_14    583
Name: acquisition, dtype: int64 

['20221117_125310_15']
20221117_125310_15    284
Name: acquisition, dtype: int64 

['20221117_

In [63]:
def plot_image(data) :
    plt.plot(data.index,data.TBS_1.cumsum(),'bo')
    plt.plot(data.index,data.TBS_1.cumsum())

In [64]:
for val in tqdm(data2.acquisition.unique()):
    current_data = data2[data2.acquisition==val]
    label = current_data.label.value_counts(sort = True, ascending =False).index[0]
    #print(label)
    plot_image(current_data)
    name_img = f"{val}.png"
    path = os.path.join(name_dataset, label, name_img )
    #print(path)
    plt.savefig(path, format = "jpg")
    plt.close()
    
    

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  2.04it/s]
